#### Email simples

In [1]:
# PESQUISAR
# https://realpython.com/python-send-email/

# Importação das bibliotecas
import pandas as pd
import win32com.client as win32
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)

In [3]:
# 
var1 = "SORRIA"
mail.To = "daniel_ijano@hotmail.com"
mail.Subject = "Assunto"
mail.HTMLBody = f"""
<p>Bodia raio disol
{var1}
<p>:)
"""

# anexo = "C://Users/blablabla/arquivo.file"
# email.Attachments.Add(anexo)

#mail.Send()

#### Iterando

In [9]:
# Create the connection with Google Cloud
def create_connection():
        # Set the credentials and create the connection
        key_path = "../usp-mba-dsa-tcc-4277103d9155.json"

        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes=["https://www.googleapis.com/auth/bigquery"]
        )

        client = bigquery.Client(
            credentials=credentials,
            project=credentials.project_id,
        )
        return client

client = create_connection()

In [18]:
# Perform a query.
QUERY = (
    "SELECT * FROM `usp-mba-dsa-tcc.ecommerce_offers.vw_send_alerts` WHERE Status='Active'"
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df = rows.to_dataframe()

Forbidden: 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

Location: southamerica-east1
Job ID: 604c36ae-88fa-4c2a-a6dd-31501dc8ce98


In [12]:
df.head(5)

,offer_id,Loja,Especie,Categoria,URL,SKU,Titulo,Descricao,rating,regular_price,sub_price,Marca,Grupo,QTD_Oferta,Embalagem_kg,Embalagem,Total_Oferta_kg
0,b'\xd5\x87\xd5\xab\xce\xa3\xf7Fw\x0e\x12w4\xdd...,Cobasi,Cachorro,Racao Umida,https://www.cobasi.com.br/racao-umida-para-cae...,31000322,Ração Úmida para Cães Adultos Joy Origem Natur...,Ração Úmida para Cães Adultos Joy Origem Natur...,5.0,22.9,21.00,Origem Natural,Adultos,1.0,0.30,300g,0.30
1,b'%\xe19\xcd\xea\xae\xd9\x98\x82n\xa6\xf4M\x9f...,Cobasi,Cachorro,Racao Umida,https://www.cobasi.com.br/racao-umida-para-cae...,31000330,Ração Úmida para Cães Filhotes Joy Origem Nature,Ração Úmida para Cães Filhotes Joy Origem Natu...,0.0,18.9,17.01,Origem Natural,Filhotes,1.0,0.30,300g,0.30
2,"b'q^\xc8\xb1\x10\xddAF,\xb6\x90U\xe3\xeb!i\x80...",Cobasi,Cachorro,Racao Umida,https://www.cobasi.com.br/racao-umida-para-cae...,31000349,Ração Úmida para Cães Sênior Joy Origem Nature,Ração Úmida para Cães Sênior Joy Origem Nature...,4.5,18.9,17.01,Origem Natural,Sênior,1.0,0.30,300g,0.30
3,"b""O;mO\x83\x14\x8f~d\xb7\xac*9\xa7\x1f\xab\xec...",Cobasi,Cachorro,Racao Umida,https://www.cobasi.com.br/racao-umida-joy-pate...,31000357,Ração Úmida Joy Patê para Cães Adultos Carne e...,Ração Úmida Joy Patê para Cães Adultos Carne e...,0.0,8.9,8.01,Joy,Adultos,1.0,0.28,280g,0.28
4,"b'\xaeJ\xcc\x08\x88,p\xcf\x8c\xe7\x7f\xe4\xc4\...",Cobasi,Cachorro,Racao Umida,https://www.cobasi.com.br/racao-umida-joy-pate...,31000381,Ração Úmida Joy Patê para Cães Adultos Carne,Ração Úmida Joy Patê para Cães Adultos Carne 2...,4.5,8.9,8.01,Joy,Adultos,1.0,0.28,280g,0.28


In [ ]:
outlook = win32.Dispatch('outlook.application')

for row in df.index[:5]:
    mail = outlook.CreateItem(0)
    mail.To = "daniel_ijano@hotmail.com"
    mail.Subject = "Categorias e Marcas"
    mail.HTMLBody = f"""
    <p>{df['Categoria'][row]}
    <p>{df['Marca'][row]}
    <p>:)
    """
    mail.Send()

# https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/